<a href="https://colab.research.google.com/github/srdg/unarchived_ben_tess/blob/main/src/Bengali_Typeset_With_Tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E2E Training Pipeline
Installs dependencies, language packs;
Gets training tools and training data 
and runs training to make `.traineddata` model. Logs final result after training is completed.

In [ ]:
!echo "*******************Install dependencies***************************"
!sudo apt-get install tesseract-ocr bc libtesseract-dev 
!echo "********************Verify installation**************************"
!which tesseract
!echo "*******************Install language***************************"
!sudo apt-get install tesseract-ocr-ben
!echo "*********************Verify language*************************"
!ls /usr/share/tesseract-ocr/4.00/tessdata | grep traineddata
!echo "******************Download training data********************"
!rm -rf ./*
!mkdir -p ./data/ben-ground-truth
!wget https://github.com/srdg/unarchived_ben_tess/\
releases/download/v0.0.3-alpha/data.zip -q --show-progress 
!unzip -q ./data.zip -d ./data/ben-ground-truth && rm -rf ./data.zip
!echo "******************Download training tools********************"
!wget https://github.com/tesseract-ocr/tesstrain/\
archive/master.zip -q --show-progress && unzip -q ./master.zip 
!rm -rf ./master.zip ./sample_data
!echo "***********************Move training  data***********************"
!mv ./data ./tesstrain-master/
!echo "======================= READY TO TRAIN ========================="
%cd ./tesstrain-master/
!echo "======================= TRAINING MODEL ========================="
!make training MODEL_NAME=ben RATIO_TRAIN=0.80 \
MAX_ITERATIONS=99999999&>./data/ben.training.log
%cd ..
!echo "======================= TRAINING DONE! ========================="
!tail -10 ./data/ben.training.log

*******************Install dependencies***************************
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  bc libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 14 not upgraded.
Need to get 7,636 kB of archives.
After this operation, 29.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 bc amd64 1.07.1-2 [86.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libleptonica-dev amd64 1.75.3-3 [1,308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtesseract-dev amd64 4.00~git2288-10f4998a-2 [1,447 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]


# Data Cleaning Pipeline
Gets data (needs to be uploaded first) and performs hocr ops on it to split images into line-level, converts into .tif format and zips them for download.  
#### Preconditions
+ Images must be in .PNG format
+ Images must be in a zipped archive without any hierarchy


In [ ]:
!wget https://github.com/ocropus/hocr-tools/archive/master.zip \
-q --show-progress && unzip master.zip -d . 
!wget https://raw.githubusercontent.com/srdg/\
unarchived_ben_tess/main/src/line_hocr.sh && chmod +xxx *.sh
!apt install tesseract-ocr-ben
!bash line_hocr.sh
%cd data
!rm -rf *.exp0.txt
!mkdir -p data
!mv *.exp0.tif ./data
!zip data.zip ./data/* -q